In [ ]:
import os

import tempfile

import numpy as np

import astropy.units as u

import toast

from toast import qarray as qa

from toast import config as tc

from toast import (
    Telescope, 
    Focalplane, 
    Observation, 
)

import toast.future_ops as ops

from toast.future_ops.sim_focalplane import fake_hexagon_focalplane


# Data Model

The basic data model is a set of `Observation` instances, each of which is associated with a `Focalplane` on a `Telescope`.  Note that a Focalplane instance is probably just a sub-set of detectors on the actual physical focalplane.  These detectors must be co-sampled and likely have other things in common (for example, they are on the same wafer or are correlated in some other way).  For this example, we will manually create these objects, but usually these will be loaded / created by some experiment-specific function.

MPI is optional in TOAST, although it is required to achieve good parallel performance on traditional CPU systems.  In this section we show how interactive use of TOAST can be done without any reference to MPI.  In a later section we show how to make use of distributed data and operations.

In [ ]:
# help(Observation)

In [ ]:
# help(Focalplane)

In [ ]:
# help(Telescope)

In [ ]:
# Start by making a fake focalplane

focalplane_pixels = 7 # (hexagonal)
field_of_view = 10.0 # degrees
sample_rate = 10.0 # Hz

focalplane = fake_hexagon_focalplane(
    focalplane_pixels,
    field_of_view,
    samplerate=10.0,
    epsilon=0.0,
    net=1.0,
    fmin=1.0e-5,
    alpha=1.0,
    fknee=0.05,
)

In [ ]:
# Now make a fake telescope

telescope = Telescope(name="fake", focalplane=focalplane)

In [ ]:
# Make an empty observation

samples = 10

obs = Observation(telescope, name="2020-07-31_A", samples=samples)

print(obs)

## Metadata

By default, the observation is empty.  You can add arbitrary metadata to the observation- it acts just like a dictionary.

In [ ]:
hk = {
    "Temperature 1": np.array([1.0, 2.0, 3.0]),
    "Other Sensor": 1.2345
}

obs["housekeeping"] = hk

print(obs)

## Time Ordered Data

Now we can add some Time Ordered Data to this observation.  There are basically two types of data:  timestreams of information that all detectors have in common (telescope boresight, etc) and timestreams of detector data (signals and flags).  Although an Observation acts like a dictionary that can hold arbitrary keys, there are some standard built-in names for TOD quantities that are used by the Operator classes.  You can also create other custom types of data.  To see the built-in names, you can do:

In [ ]:
print(obs.keynames)

These underlying names can be overridden at construction time if you like.

### Time Ordered Detector Data

Detector data has some unique properties that we often want to leverage in our analyses.  Each process has some detectors and some time slice of the observation.  In the case of a single process like this example, all the data is local.  Before using data we need to create it within the empty `Observation`.  Here we create the default SIGNAL data:

In [ ]:
obs.create_signal()

In [ ]:
print(obs.signal)

This special `DetectorData` class is a table that can be indexed either by name or by index.  You can set and get values as usual:

In [ ]:
obs.signal["D0A"] = np.arange(samples, dtype=np.float64)

obs.signal[1] = 10.0 * np.arange(samples, dtype=np.float64)

print(obs.signal)

In [ ]:
print(obs.signal[:])

In [ ]:
print(obs.signal["D0A", "D0B"])

In [ ]:
print(obs.signal[1][1:5])

This showed the creation of the default "SIGNAL" detector data, but you can create other types of data.  For example, lets say you wanted to create some detector pointing matrix values consisting of a 64bit integer pixel number and three 32bit floats for the I/Q/U weights:

In [ ]:
obs.create_detector_data("pixels", shape=(samples,), dtype=np.int64)
obs.create_detector_data("weights", shape=(samples, 3), dtype=np.float32)

print(obs["weights"])

In [ ]:
weights = obs["weights"]

for d in obs.detectors:
    for s in range(samples):
        weights[d][s] = [1.0, 0.5, 0.5]
        
print(obs["weights"])

### Time Ordered Data Shared by all Detectors

There are some types of timestreams which all detectors have in common within the observation.  These include things like telescope pointing, timestamps, and other quantities.  We want all processes to have access to these quantities.  However, this type of data is usually stored once and then read many times.  We use shared memory on the node to store this data to avoid duplicating it for every process.  For this simple serial example, the details are not important.  The main thing is to use a special method when creating these buffers in the observation.  For example:

In [ ]:
obs.create_times()
obs.create_boresight_radec()
obs.create_common_flags()


In [ ]:
# This accesses the timestamps regardless of the underlying
# dictionary key and checks that the underlying buffer has
# the right dimensions.

print(obs.times)

In [ ]:
print(obs.times[0:5])

The `create_*()` methods create these shared memory objects of the correct default dimensions and type.  You can also create completely custom timestream data (see advanced topics below).

After creating the shared buffer, we used the observation method `times()` to return the timestamps.  There are similar methods for all the "standard" observation data products (boresight_radec(), signal(), etc).  The benefit to using these methods instead of accessing the internal dictionary key directly is that there are checks on the shapes of the underlying objects to ensure consistency.  Also, an operator does not have to know the name of the underlying dictionary key, which might be different between experiments.

These shared data objects have a `set()` method used to write to them.  This is more important when using MPI.  In the serial case, you can just do:

In [ ]:
obs.times.set(np.arange(samples, dtype=np.float64))

print(obs.times[:])

# Data

The `Observation` instances discussed previously are usually stored as a list inside a top-level container class called `Data`.  This class also stores the TOAST MPI communicator information.  For this serial example you can just instantiate an empty `Data` class and add things to the observation list:

In [ ]:
data = toast.Data()

print(data)

print(data.obs)

Obviously this `Data` object has no observations yet.  We'll fix that in the next section!

# Processing Model

The `Operator` class defines the interfaces for operators working on data.  Each operator constructor takes only keyword arguments, and these keyword arguments are stored as class attributes in the instance.  Each operator has methods that describe the observation dictionary keys it requires for input and which keys it provides as output.  An operator has an `exec()` method that works with `Data` objects.  We will start by looking at the `SimSatellite` operator to simulate fake telescope scan strategies for a generic satellite.  We can always see the options and default values by using the standard help function or the '?' command:

In [ ]:
help(ops.SimSatellite)

?ops.SimSatellite

You can instantiate a class directly by overriding some defaults:

In [ ]:
simsat = ops.SimSatellite(
    n_observation=2, 
    observation_time=(5 * u.minute),
)

print(simsat)

After the operator is constructed, the parameters can be changed directly.  For example:

In [ ]:
simsat.telescope = telescope
simsat.n_observation = 3

In [ ]:
print(simsat)

And now we have an `Operator` that is ready to use.  This particular operator creates observations from scratch with telescope properties generated and stored.  We can create an empty `Data` object and then run this operator on it:

In [ ]:
data = toast.Data()

In [ ]:
simsat.exec(data)
simsat.finalize(data)

In [ ]:
print(data)

In [ ]:
data.info()

In [ ]:
print("There are {} observations".format(len(data.obs)))

In [ ]:
print(data.obs[0])

In [ ]:
for ob in data.obs:
    print(ob.times[:5])
    print(ob.boresight_radec[:5])

So we see that our `SimSatellite` operator has created just one observation of 360 samples in the `Data` object.  We can feed this tiny dataset to further operators to simulate signals or process the data.  Let's now simulate some noise timestreams for our detectors.  First we need to create a "noise model" for our detectors.  We can bootstrap this process by making a noise model from the nominal detector properties in the focalplane:

In [ ]:
ops.DefaultNoiseModel.help()

In [ ]:
noise_model_config = ops.DefaultNoiseModel.defaults()
print(noise_model_config)

noise_model = ops.DefaultNoiseModel(noise_model_config)
noise_model.exec(data)
noise_model.finalize(data)

Now we are ready to use the `SimNoise` operator to simulate some timestreams:

In [ ]:
ops.SimNoise.help()

In this case, we can just use all the default options.  It assumes the default noise model and if we don't specify the `out` key this operator just accumulates to the default detector data ("SIGNAL").

In [ ]:
noise_config = ops.SimNoise.defaults()
print(noise_config)

In [ ]:
# Create the operator

sim_noise = ops.SimNoise(noise_config)

In [ ]:
# Run it on the data

sim_noise.exec(data)
sim_noise.finalize(data)

In [ ]:
data.info()

Notice that the observation now has some signal.  Let's look at that:

In [ ]:
# print(data.obs[0].signal())

In [ ]:
# Just look at few samples from one detector in the first observation

print(data.obs[0].signal["D1A"][:5])

## Pipeline Operator

There is a special Operator class called `Pipeline` which serves as a way to group other operators together and run them in sequence (possibly running them on only a few detectors at a time).  The default is to run the list of operators on the full `Data` object in one shot.  The Pipeline class has the usual way of getting the defaults:

In [ ]:
ops.Pipeline.help()

We'll see more about this Operator below.

## Configuration Files

We saw above how operators are constructed with a dictionary of parameters.  **You can do everything by passing parameters when constructing operators**.  Configuration files are completely optional, but they do allow easy sharing of complicated pipeline setups.

These parameters can be loaded from one or more files and used to automatically construct operators for use.  When doing this, each instance of an operator is given a "name" that can be used to reference it later.  This way you can have multiple operators of the same class doing different things within your pipeline.  If you have a script where you know which operators you are going to be using, you can get the defaults for the whole list at once:

In [ ]:
pipe_ops = {
    "sim_satellite": ops.SimSatellite,
    "noise_model": ops.DefaultNoiseModel,
    "sim_noise": ops.SimNoise,
    "pointing": ops.PointingHealpix
}

conf = default_config(operators=pipe_ops)

print(conf)

We can dump this to a file and look at it:

In [ ]:
tmpdir = tempfile.mkdtemp()
conf_file = os.path.join(tmpdir, "test.toml")

dump_config(conf_file, conf)

In [ ]:
 !cat {conf_file}

... and we can also load it back in:

In [ ]:
newconf = load_config(conf_file)

print(newconf)

What if we wanted to add a Pipeline to this configuration that reference the names of the two operators to use?  We can do that using a special syntax which consists of `@config:` followed by a UNIX-style path to the object we are referencing.  For example:

In [ ]:
# Get the default Pipeline config

sim_pipe_config = ops.Pipeline.defaults()
print(sim_pipe_config)

In [ ]:
# Add references to the operators

sim_pipe_config["operators"] = [
    "@config:/operators/sim_satellite",
    "@config:/operators/noise_model",
    "@config:/operators/sim_noise",
    "@config:/operators/pointing",
]

# Add the pipeline config to the main config

newconf["operators"]["sim_pipe"] = sim_pipe_config

print(newconf)

Now we could dump this to a file for later use.  What if we wanted to go and create operators from this?  We could loop through each key in the "operators" dictionary and instantiate the class with the config values.  However, there is a helper function that does this.  Before doing that we need to add a Telescope to this config for the satellite simulation.  Normally this would be done by some experiment-specific script that would create a more custom telescope / focalplane.

In [ ]:
newconf["operators"]["sim_satellite"]["telescope"] = telescope

Now instantiate all the operators in one go:

In [ ]:
run = create(newconf)

print(run)

Although the result looks similar, look more closely.  Our dictionary of configuration options is now actually a dictionary of instantiated classes.  We can now run these operators directly:

In [ ]:
data = toast.Data()

# Run the Pipeline operator, which in turn runs 2 other operators

run["operators"]["sim_pipe"].exec(data)

In [ ]:
data.info()

In [ ]:
# Print the first 5 samples of one detector in the first observation.

print(data.obs[0].signal["D0A"][:5])

In [ ]:
print(data.obs[0]["weights"]["D0A"][:5])

# Advanced Topics

The previous sections covered the `Observation` container and its interfaces, and how to create and run Operators on a `Data` object containing a list of observations.  The new data model has some aspects that improve our situation on larger runs.

## Memory Use

Earlier we saw how the MPI shared memory objects created in an Observation are used to store things that are common to all detectors (boresight pointing, telescope velocity, etc).  These quantities have defaults for the shape, dtype, **and** the communicator used.  In the case of these common properties, the "grid column communicator" is used.  This includes all processes in the observation that share a common slice of time.  The result is that only one copy of these common data objects exist on each node, regardless of how many processes are running on the node.

However, we can create completely custom shared memory objects.  Imagine that every single process needed some common telescope data to be able to work with its local signal.  We could create a shared object on the group communicator used for the whole observation:

In [ ]:
samples = 10

obs = Observation(telescope, name="2020-07-31_A", samples=samples)

# This is the same for every process, regardless of location in the process grid

obs.create_shared_data(
    "same_for_every_process", 
    shape=(100, 100), 
    dtype=np.float64, 
    comm=obs.mpicomm
)

In another example, suppose that we had some detector-specific quantities (beams, bandpasses, etc) shared by all processes with data from a given detector.  We can store that in shared memory using the "grid row communicator" of the process grid, so that we only have one copy of those products per node:

In [ ]:
# This is the same for every process along a row of the grid
# so these processes all have the same detectors.

obs.create_shared_data(
    "detector_aux_data", 
    shape=(len(obs.local_detectors), 100), 
    dtype=np.float64, 
    comm=obs.grid_comm_row
)

The use of MPI shared memory should greatly reduce our memory footprint when running many MPI processes per node.

## Processing Subsets of Detectors

The `Pipeline` operator is used to chain other operators together and can internally feed data to those sub-operators in sets of detectors.  This is a work in progress, but the workflow code that creates the `Pipeline` will be able to specify sets of detectors to process at once.  This set will be different on different processes.  The special strings "ALL" and "SINGLE" are used to either work with all detectors in one shot (the current toast default) or to loop over detectors individually, running all operators on those before moving on to the next.

## Accelerator Use

This covers planned features...

For each supported architecture, if all operators in a pipeline support that hardware then the pipeline can use observation methods to copy select data to the accelerator at the beginning and back from the accelerator at the end.  Operators individually have methods that specify the observation keys they "require" and also the observation keys they "provide".  This allows logic in the pipeline operator to determine which intermediate data products of the operators are only on the accelerator and do not need to be moved back to the host system.  A Pipeline running on an accelerator will likely process only a few detectors at a time due to memory constraints.

Observations already make use of MPI shared memory that is replicated across nodes.  Each node will have some number of accelerators.  We can assign each process to a particular accelerator and compute the minimal set of shared memory objects that need to be staged to each accelerator.